In [1]:
from trueskill import Rating, rate_1vs1, quality_1vs1, BETA
from trueskill.backends import cdf
import trueskill
from math import sqrt, pow
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.style.use(['ggplot', 'grayscale'])
%matplotlib inline

trainingSet = "train_set.txt"
testSet = "test_set.txt"

players = {}

trainDF = pd.read_csv(trainingSet, header=None, sep="\t")
testDF = pd.read_csv(testSet, header=None, sep="\t")

# trainDF = trainDF.iloc[np.random.permutation(len(trainDF))]
# trainDF = trainDF.reset_index(drop=True)

In [3]:
trueskill.__version__

'0.4'

In [2]:
def win_probability(rA=Rating(), rB=Rating()):
    deltaMu = rA.mu - rB.mu
    sumSigma = rA.sigma ** 2 + rB.sigma ** 2
    playerCount = 2
    denominator = sqrt(playerCount * (BETA * BETA) + sumSigma)
    return cdf(deltaMu / denominator)

In [3]:
def experiment(env=trueskill.TrueSkill(draw_probability=0.0)):
    for name in pd.concat([trainDF[0], trainDF[1], testDF[0], testDF[1]]).unique():
        players[name] = env.create_rating()

    for row in trainDF.itertuples():
        i, winner, loser  = row
        players[winner], players[loser] = rate_1vs1(players[winner], players[loser], env=env)

    rows = []
    for row in testDF.itertuples():
        i, p1, p2, actual  = row
        pred = win_probability(players[p1], players[p2])
        rows.append([p1, p2, 1-pred, actual])
    rows = pd.DataFrame(rows, columns=['a', 'b', 'solve_prob', 'actual'])

    return rows

In [4]:
def score(df, eps=1e-19):
    return -np.mean(np.log(np.abs(df['solve_prob'] - df['actual']).clip(eps, 1-eps)))
def mae(df):
    return np.mean(np.abs(df['solve_prob'] - df['actual']))
def rmse(df):
    return np.sqrt(np.mean((df['solve_prob']-df['actual'])**2))
def crossentropy(df):
    return -np.mean(df['actual'] * np.log(df['solve_prob']) + (1-df['actual']) * (1-np.log(df['solve_prob'])))

In [5]:
def hc_acc_rmse(df, adqs=None):
    reshptest = pd.read_csv("res.hp.test.txt", sep="\t", header=None)
    reshptest.columns = ["pid", "handle", "solved", "contest", "level"]
    reshptest['solve_prob'] = df['solve_prob']
    reshptest.set_index(['contest', 'handle'])

    adqs = []
    for (contest, handle), rows in reshptest.groupby(['contest', 'handle']):
        cum = 1
        adq_probs = []
        for i, pid, handle, solved, contest, level, solve_prob in rows[::-1].itertuples():
            adq_probs.append(cum * solve_prob)
            cum = cum * (1-solve_prob)
        adq_probs.append(cum)
        adq = np.argmax(adq_probs[::-1])
        adqs.append((contest,handle,adq))
    adqs = pd.DataFrame(adqs)
    adqs.columns = ["contest", "handle", "pred"]

    hc_test_actual = pd.read_csv("res.hc.test.txt", header=None, sep="\t")
    hc_test_actual.columns = ["handle", "contest", "actual"]
    
    merged = adqs.merge(hc_test_actual, left_on=("contest", "handle"), right_on=("contest", "handle"))
    
    return (merged['actual'] == merged['pred']).mean(), np.sqrt(((merged['actual'] - merged['pred'])**2).mean())


In [6]:
# mu = 25.0
# sigma = mu / 3.0
# beta = sigma / 0.5
# tau = sigma / 100
# model = experiment(trueskill.TrueSkill(mu=mu, sigma=sigma, beta=beta, tau=tau))
model = experiment()

In [16]:
model.head()

,a,b,solve_prob,actual
0,101_A,Alias,0.996193,1
1,101_B,Alias,0.682612,1
2,101_C,Alias,0.262490,1
3,101_D,Alias,0.075249,0
4,101_E,Alias,0.015137,0


# Trueskill 0.4.4

In [7]:
# Trueskill 0.4.4
"""
3.14182699133
0.205342760137
0.326424958383
-3.01832813703
(0.47316538882803943, 1.1224744480187112)
"""
print score(model)
print mae(model)
print rmse(model)
print crossentropy(model)
print hc_acc_rmse(model)

3.14182699133
0.205342760137
0.326424958383
-3.01832813703
(0.47316538882803943, 1.1224744480187112)


In [7]:
# trueskill 0.4
"""
3.14182699133
0.205342760137
0.326424958383
-3.01832813703
(0.47316538882803943, 1.1224744480187112)
"""
print score(model)
print mae(model)
print rmse(model)
print crossentropy(model)
print hc_acc_rmse(model)

3.14182699133
0.205342760137
0.326424958383
-3.01832813703
(0.47316538882803943, 1.1224744480187112)


In [7]:
# trueskill 0.2
"""
3.14182699133
0.205342760137
0.326424958383
-3.01832813703
(0.47316538882803943, 1.1224744480187112)
"""
print score(model)
print mae(model)
print rmse(model)
print crossentropy(model)
print hc_acc_rmse(model)

3.14182699133
0.205342760137
0.326424958383
-3.01832813703
(0.47316538882803943, 1.1224744480187112)


In [11]:
trueskill.

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=10.0%)

# Parameter Tuning

In [13]:
# Don't initialize it, if you want to cumulate the result for param tuning
param_model = {}

In [14]:
for bd in [1.2, 1.4, 1.6, 1.8]:
    for td in [1e4, 1e5, 1e6, 1e7]:
        if (bd,td) in param_model:
            continue
        mu = 25.0
        sigma = mu / 3.0
        beta = sigma / bd
        tau = sigma / td
        model = experiment(trueskill.TrueSkill(mu=mu, sigma=sigma, beta=beta, tau=tau, draw_probability=0))
        param_model[(bd,td)] = model
        print ((bd,td), rmse(model)) 

((1.2, 10000.0), 0.33143365859799617)
((1.2, 100000.0), 0.33143366539749564)
((1.2, 1000000.0), 0.33143366546553582)
((1.2, 10000000.0), 0.33143366546621633)
((1.4, 10000.0), 0.3289348445675549)
((1.4, 100000.0), 0.32893484644530918)
((1.4, 1000000.0), 0.32893484646415644)
((1.4, 10000000.0), 0.32893484646434562)
((1.6, 10000.0), 0.32730902947153101)
((1.6, 100000.0), 0.32730902485862046)


KeyboardInterrupt: 

In [15]:
pic = []
for (bd,td) in param_model:
    hp_score  = score(param_model[(bd,td)])
    hp_mae  = mae(param_model[(bd,td)])
    hp_rmse = rmse(param_model[(bd,td)])
    hp_ce = crossentropy(param_model[(bd,td)])
    hc_acc, hc_rmse = hc_acc_rmse(param_model[(bd,td)])

    pic.append((bd, td, hp_score, hp_mae, hp_rmse, hp_ce, hc_acc, hc_rmse))

pic = pd.DataFrame(pic)

/home/nicola/anaconda/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [16]:
# SCORE: 2
# MAE: 3
# RMSE: 4
# CROSS_ENTROPY: 5
# HC ACCURACY: 6
# HC RMSE: 7

pic_cut = pic
# pic_cut = pic_cut[pic_cut[6] > .47]
# pic_cut = pic_cut[pic_cut[7] < 1.1]
# pic_cut[6] = pic_cut[6].clip(0.47,1)
# pic_cut[7] = pic_cut[7].clip(0,1.09)

f, axes = plt.subplots(6)
for i in range(6):
    pic_cut.plot.scatter(x=0,y=1,c=(i+2),s=50, figsize=(12, 20), title=["SCORE", "MAE", "RMSE", "CROSS_ENTROPY", "HC ACCURACY", "HC RMSE"][i], ax=axes[i],logy=True, xlim=(0,3))

In [285]:
m = param_model[(1.4, 1e3)]
m.loc()[np.argsort(np.abs(m['solve_prob'] - m['actual']))[::-1]]

,a,b,solve_prob,actual
5490,145_A,Dmitry_Egorov,9.999999e-01,0
435,103_A,ikatanic,9.999990e-01,0
784,107_E,xiaowuc1,6.685029e-06,1
28404,372_E,permin,1.265843e-05,1
38956,77_B,tourist,9.999859e-01,0
3999,132_E,K_operafan,1.859919e-05,1
33266,407_A,rng_58,9.999783e-01,0
34533,418_D,laoriu,2.305424e-05,1
37445,446_A,Petr,9.999767e-01,0
4603,135_D,johnLate,2.694948e-05,1


In [301]:
model.head(20)

,a,b,solve_prob,actual
0,101_A,Alias,9.999923e-01,1
1,101_B,Alias,5.372282e-01,1
2,101_C,Alias,8.824241e-03,1
3,101_D,Alias,1.744933e-05,0
4,101_E,Alias,6.442600e-08,0
5,101_A,Alireza,9.997641e-01,1
6,101_B,Alireza,2.012649e-01,0
7,101_C,Alireza,4.062805e-04,0
8,101_D,Alireza,1.412410e-07,0
9,101_E,Alireza,1.674855e-10,0


In [299]:
param_model[(0.5, 100)]

,a,b,solve_prob,actual
0,101_A,Alias,9.999996e-01,1
1,101_B,Alias,7.948509e-01,1
2,101_C,Alias,5.575133e-02,1
3,101_D,Alias,1.876034e-04,0
4,101_E,Alias,1.679924e-06,0
5,101_A,Alireza,9.999829e-01,1
6,101_B,Alireza,4.578974e-01,0
7,101_C,Alireza,5.062448e-03,0
8,101_D,Alireza,2.478372e-06,0
9,101_E,Alireza,7.518410e-09,0


In [304]:
mae(param_model[(1.4,1e3)])

0.18330756577364493